In [1]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

In [2]:
def read_text_file(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()

In [3]:
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

walletAlice = Wallet()
walletBob = Wallet()

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)

In [8]:
import time
from certificate import Certificate
from helpers import timestamp
from smart_contracts.marketplace import SmartContract
from smart_contract import SmartContractDefinition, SmartContractWritingOperation

reset_blockchain()

smartContractDefinition = SmartContractDefinition(walletAlice.publicKey, read_text_file('smart_contracts/marketplace.py'), [])
walletAlice.sign(smartContractDefinition)

node1.new_certificate(smartContractDefinition)

aliceSellOperation1 = SmartContract.add_item_certificate(walletAlice.publicKey, smartContractDefinition, SmartContract.ItemInfo("Table","Nice wooden table",  100))

aliceSellOperation2 = SmartContract.add_item_certificate(walletAlice.publicKey, smartContractDefinition, SmartContract.ItemInfo("Chair","Nice wooden chair",  50))

walletAlice.sign(aliceSellOperation1)
walletAlice.sign(aliceSellOperation2)

node1.new_certificate(aliceSellOperation1)
node1.new_certificate(aliceSellOperation2)

######################

for i in range(10):
    certificate = Certificate(walletAlice.publicKey)
    
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

######################
# node2.blockchain.display()

smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node2.blockchain,
    smartContractDefinition.hash()
)


smartContractObject.display()

Market items: [('Table', 'Nice wooden table', 100, '4ea0a3fa'), ('Chair', 'Nice wooden chair', 50, '4ea0a3fa')]


In [12]:
bobTransferOperation = SmartContract.transfer_certificate(walletBob.publicKey, smartContractDefinition, walletAlice.publicKey, 2)

walletAlice.sign(bobTransferOperation)

node2.new_certificate(bobTransferOperation)

bobTransferOperation = SmartContract.transfer_certificate(walletBob.publicKey, smartContractDefinition, walletBob.publicKey, 2)

walletBob.sign(bobTransferOperation)

node2.new_certificate(bobTransferOperation)

bobTransferOperation = SmartContract.transfer_certificate(walletBob.publicKey, smartContractDefinition, walletAlice.publicKey, 2)

walletBob.sign(bobTransferOperation)

node2.new_certificate(bobTransferOperation)


######################

for i in range(10):
    certificate = Certificate(walletAlice.publicKey)
    
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

######################

smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node2.blockchain,
    smartContractDefinition.hash()
)


smartContractObject.display()


Minting is not allowed at this time
You are already the owner of this token
SmartContract NFT:
Token owners: ['1: 2b95d86d', '2: 2b95d86d']
Start mint timestamp: 1740650186858
Mint time: 200
Token id: 2


In [13]:
from flask import Flask
from dominate import document
from dominate.tags import *

app = Flask(__name__)

@app.route('/')
def home():
    doc = document(title="My Page")
    with doc.head:
        title("My Page")
        script(src="https://unpkg.com/htmx.org@2.0.4", integrity="sha384-HGfztofotfshcF7+8n44JQL2oJmowVChPTg48S+jvZoztPfvwD79OC/LTtG6dMp+", crossorigin="anonymous")
    with doc:
        with body():
            h1("Hello, World!")
            b = button("Click me")
            b["hx-get"] = "/a"
            div().add_raw_string(smartContractObject.html())
            
    return doc.render()

@app.route('/a')
def a():
    return "You clicked the button!"

if __name__ == '__main__':
    app.run(port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
